In [0]:
import numpy
import keras
import pandas
import requests
import io
import zipfile
import os
import re
import cleverhans
import tensorflow

from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks_tf import jacobian_augmentation
from cleverhans.attacks_tf import jacobian_graph
from cleverhans.loss import CrossEntropy
from cleverhans.train import train
from cleverhans.utils_keras import KerasModelWrapper

from keras.models import Sequential
from keras.layers import Dense

numpy.random.seed(0xC0FFEE)
tensorflow.set_random_seed(0xC0FFEE)
rng = numpy.random.RandomState(0xC0FFEE)

# Dataset

## Loading data

Run the below code to download a copy of the dataset (if you don't already have it):

In [0]:
response = requests.get("http://www.schonlau.net/masquerade/masquerade-data.zip")

dataset_file = io.BytesIO(response.content)

zipped_dataset = zipfile.ZipFile(dataset_file)
zipped_dataset.extractall('data/masquerade-data')

In [0]:
# http://www.schonlau.net/intrusion.html
# download Masquerade Data (zip File)

import pandas as pd
directory = './data/masquerade-data'

In [0]:
def sorted_nicely( l ):
    """ Sorts the given iterable in the way that is expected.
 
    Required arguments:
    l -- The iterable to be sorted.
 
    """
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [0]:
users = range(1,51)
df = pd.DataFrame()

for filename in sorted_nicely(os.listdir(directory)):
    user = pd.read_csv(os.path.join(directory, filename), header=None)
    df = pd.concat([df, user], axis = 1)
    
df.columns = sorted_nicely(os.listdir(directory))

We've loaded in the dataset, but need to do a little co-ercion to get it how we need. Firstly, make sure that all the values in this dataframe are categorical variables which share the same data type:

In [0]:
commands = numpy.unique(df)
command_dtype = pandas.api.types.CategoricalDtype(commands)

for column in df:
    df[column] = df[column].astype(command_dtype)

In [0]:
labelled, unlabelled = df.head(5000), df.tail(len(df) - 5000)  # ignore unlabeled

Plan is convert to the following format:

  user, command1?, command2?, ..., 
  
 so the first column is a label, and the second a one-hot encoding of the command.
 
 When we do the rolling window aggregation, we just sum the columns (per-user).
 
 Use [rolling window sampling](https://pcp.io/books/PCP_PG/html/LE42586-PARENT.html).

In [12]:
def rolling_window_command_counts(commands, window_size):
    
    # Save a copy the name of the series to add again to our output. This will preserve the mapping of
    # user identifier to (it's column header in the dataframe it came from), which in
    # this case is the user identifier. 
    user = commands.name

    # Convert the single column "which command was run?" to a column for each
    # command, which says "was command <x> run?"
    commands = pandas.get_dummies(commands)

    # Take a rolling sample of the last 100 commands, then sum each "was command <x> run?"
    # columns to give a bunch "command <x> was run <y> times in this window".
    command_counts = commands.rolling(window=window_size).aggregate(numpy.sum)

    # Remove the first 100 rows because they contain data from blocks of size < 100.
    command_counts = command_counts[window_size-1:]
    
    # Preserve the user identifier (see top of function) as a new column:
    
    # First, a nasty hack: https://github.com/pandas-dev/pandas/issues/19136
    command_counts = command_counts.rename(columns=str)  
    
    # Then, add in the user (with an adhoc parser to turn the label into a number)
    command_counts['user'] = int(user.replace('User', ''))

    return command_counts

# Example
rolling_window_command_counts(labelled['User1'], 100)

,%backup%,.java_wr,.maker_w,.wrapper,.xinitrc,.xsessio,1.1,1.2,1.3,4Dwm,...,xxx,yacc,ypcat,yppasswd,z,zip,zsh,zubs,zz2,user
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [13]:
labelled_dataset = pandas.concat([
        rolling_window_command_counts(commands, 100)
        for user, commands in labelled.iteritems()
    ],
    ignore_index=True,  # reset index to go from 0 to 4900
)

labelled_dataset

,%backup%,.java_wr,.maker_w,.wrapper,.xinitrc,.xsessio,1.1,1.2,1.3,4Dwm,...,xxx,yacc,ypcat,yppasswd,z,zip,zsh,zubs,zz2,user
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [0]:
labels = labelled_dataset['user'] - 1
dataset = labelled_dataset.drop(columns=['user'])

In [0]:
labels =  keras.utils.to_categorical(labels, num_classes=50)

In [0]:
from sklearn.model_selection import train_test_split

training_data, testing_data, training_labels, testing_labels = train_test_split(
    dataset,
    labels, 
    test_size=0.10,
#     stratify=list(range(50))
)

# Building the Oracle

In [0]:
oracle = Sequential()

In [0]:
input_layer = Dense(
    units=856,
    activation='relu',
    input_dim=856,
)

In [0]:
hidden_layer = Dense(
    units=30,
    activation='relu',
)

In [0]:
output_layer = Dense(
    units=50,
    activation='softmax',
)

In [21]:
oracle.add(input_layer)
oracle.add(hidden_layer)
oracle.add(output_layer)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
oracle.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

# Training Oracle on Dataset

In [23]:
oracle.fit(training_data,  training_labels, epochs=3, batch_size=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
220545/220545 [==============================] - 78s 355us/step - loss: 0.1867 - acc: 0.9466
Epoch 2/3
220545/220545 [==============================] - 77s 351us/step - loss: 0.0651 - acc: 0.9781
Epoch 3/3
220545/220545 [==============================] - 77s 351us/step - loss: 0.0508 - acc: 0.9824


In [24]:
oracle

# Evaluating the Oracle

In [25]:
loss, accuracy = oracle.evaluate(testing_data, testing_labels)

24505/24505 [==============================] - 2s 78us/step


In [26]:
loss, accuracy

(0.04324424029216407, 0.9835951846343015)

In [27]:
oracle.metrics_names

['loss', 'acc']

# Building a Substitute Model

Attack Model:
  - We only have access to the oracle as a black box, allowing the following interactions:
    - Send input.
    - Receive a prediction.

Need to search the input space to find the decision boundaries, use these inputs to train the substitute model. Note: we can overfit here and it's absolutely fine!

The original "Blackbox... " paper has an accompanying Python library,  [cleverhans](https://github.com/tensorflow/cleverhans/), which we use to implement this attack.

First, mirror the architecture of the oracle:
    

In [0]:
substitute = Sequential()

input_layer = Dense(
    units=856,
    activation='relu',
    input_dim=856,
)
hidden_layer = Dense(
    units=30,
    activation='relu',
)
output_layer = Dense(
    units=50,
    activation='softmax',
)

substitute.add(input_layer)
substitute.add(hidden_layer)
substitute.add(output_layer)

# We need to convert our substitute model into the cleverhans format.
substitute_ch = KerasModelWrapper(substitute)

In [0]:
tensorflow_session = tensorflow.Session()

We start by giving the adversary a small dataset with which to bootstrap it's search. Initially, we give it a random sample of 5% of the original data set. 

We can then steal the rest of the dataset to determine the accuracy of our substitute model. **NOTE** They do this in the tutorial code but, is it legit? Or are we cheating?


In [119]:
adversary_training_set, adversary_test_set = train_test_split(labelled_dataset, train_size=0.05)

adversary_training_inputs = adversary_training_set.drop('user', axis='columns')
adversary_training_labels = adversary_training_set['user'] - 1  # keras requires 0 based index

# I think we can delete this test part
# adversary_test_labels = adversary_test_set.drop('user', axis='columns')
# adversary_test_labels = adversary_test_set['user'] - 1  # keras requires 0 based index

# For some reason cleverhans doesn't detect a GPU when it runs, but our models at the top using
# keras _do_. I think this creates a type mis-match: code running on the GPU uses numpy.float64
# whilst the cleverhans stuff runs on the CPU and extects numpy.float32 (or vica versa).
#   -> This is why this dodgy type conversion exists:
adversary_training_inputs = adversary_training_inputs.values.astype(numpy.float32)
adversary_training_labels = adversary_training_labels.values

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Define input placeholders for the tensor flow model (these are then used to generate new points)

In [0]:
number_of_users = 50
number_of_commands = 856

input_placeholder = tensorflow.placeholder(
    tensorflow.float32,
    shape=(None, number_of_commands)
)

output_placeholder = tensorflow.placeholder(
    tensorflow.float32,
    shape=(None, number_of_users)
)

Get the oracles predictions for the "bootstrap" inputs:

In [0]:
bootstrap_oracle_predictions = oracle.predict(adversary_training_inputs)

Train substitute using method from https://arxiv.org/abs/1602.02697

In [0]:
# print("Training the substitute model.")
# model_sub, preds_sub  = train_sub(
#     tensorflow_session,
#     input_placeholder,
#     output_placeholder, 
#     bootstrap_oracle_predictions,  # predictions from the blackbox model for the _adversary_ training set?
#     adversary_training_inputs,
#     keras.utils.to_categorical(adversary_training_labels, num_classes=50),
#     number_of_users,  # number of outputs
#     10, # epochs
#     32, # batch size
#     0.001,  # learning rate (I copied and pasted default from here: https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer)
#     6, # Number of substitute data augmentations, 6 frm code
#     0.1,  # step-size of the jacobian augmentation, 0.1 from paper
#     512, # aug batch size
#     rng, # an rng
#     number_of_commands,  # number of input features
# )

In [0]:
# Creates the substitute by alternatively augmenting the training data and training the substitute.
#   :param sess: TF session
#   :param x: input TF placeholder
#   :param y: output TF placeholder
#   :param bbox_preds: output of black-box model predictions
#   :param x_sub: initial substitute training data
#   :param y_sub: initial substitute training labels
#   :param nb_classes: number of output classes
#   :param nb_epochs_s: number of epochs to train substitute model
#   :param batch_size: size of training batches
#   :param learning_rate: learning rate for training
#   :param data_aug: number of times substitute training data is augmented
#   :param lmbda: lambda from arxiv.org/abs/1602.02697
#   :param rng: numpy.random.RandomState instance

In [0]:
# Define the predictions and loss of the model, symbolically in TensorFlow (i.e. these variables 
# point to the result of calculations that haven't been performed yet)
substitute_predictions = substitute_ch.get_logits(input_placeholder)
substitute_loss = CrossEntropy(substitute_ch, smoothing=0)

Here we define a Jacobian Graph/Model (**TODO** What in the world is this? and why are we using it?).

In [125]:
adversary_training_inputs.shape, adversary_training_labels.shape

((12252, 856), (12252,))

In [0]:
# Define the Jacobian symbolically using TensorFlow
grads = jacobian_graph(substitute_predictions, input_placeholder, number_of_users)

number_of_dataset_augmentation_batches = 6  # copied from tutorial (may need tweaking)
dataset_augmentation_batch_size = 512  # copied from tutorial (may need tweaking)


lmbda = 1  # this is the step-size of the Jacobian augmentation (tutorial used 0.1, but we are 
                      # working in ints so try 1).


# Train the substitute and augment dataset alternatively
for batch in range(number_of_dataset_augmentation_batches):
    print("BATCH #" + str(batch))
    
    print("Substitute training epoch:")
    train(
        tensorflow_session, 
        substitute_loss,
        adversary_training_inputs, 
        keras.utils.to_categorical(adversary_training_labels, num_classes=50),
        init_all=False,
        args={
            'nb_epochs': 10,  # copied from tutorial (may need tweaking)
            'batch_size': 32,  # copied from tutorial (may need tweaking)
            'learning_rate': 0.001,  # copied from tutorial (may need tweaking)
        },
        rng=rng,
        # var_list=  # list of model parameters to train (optional so left out for now)
    )
    

    # If we are not at last substitute training iteration, augment dataset
    in_final_batch = batch == number_of_dataset_augmentation_batches - 1
    if not in_final_batch:
        print("Generating new data points:")
        
        # Use Jacobian augmentation to generate new data points:
        
        # TODO: What is this?? I think this scales importance per batch? != 0 means they are
        # alternating mod 3 I think.
        lmbda_coef = 2 * int(int(batch / 3) != 0) - 1 

        augmented_dataset_inputs = jacobian_augmentation(
            tensorflow_session, 
            input_placeholder, 
            adversary_training_inputs, 
            adversary_training_labels,
            grads,
            lmbda_coef * lmbda,
            dataset_augmentation_batch_size,
        )

        # Send the newly generated data points to the oracle, and use its output as their labels:
        augmented_dataset_labels = oracle.predict(augmented_dataset_inputs)
        just_new_labels = augmented_dataset_labels[len(adversary_training_inputs):]

        # Note here that we take the argmax because the adversary
        # only has access to the label (not the probabilities) output
        # by the black-box model
        thresholded_new_labels = numpy.argmax(just_new_labels, axis=1)

        augmented_dataset_labels = numpy.hstack([adversary_training_labels, thresholded_new_labels])

        # Replace dataset and labels with augmented dataset and labels
        adversary_training_inputs = augmented_dataset_inputs
        adversary_training_labels = augmented_dataset_labels

BATCH #0
Substitute training epoch:
num_devices:  1


/usr/local/lib/python3.6/dist-packages/cleverhans/utils_tf.py:511: UserWarning: No GPUS, running on CPU
  warnings.warn("No GPUS, running on CPU")
[INFO 2019-03-13 17:06:41,071 cleverhans] Epoch 0 took 2.271697521209717 seconds
[INFO 2019-03-13 17:06:43,228 cleverhans] Epoch 1 took 1.9716122150421143 seconds
[INFO 2019-03-13 17:06:45,363 cleverhans] Epoch 2 took 1.95900559425354 seconds
[INFO 2019-03-13 17:06:47,511 cleverhans] Epoch 3 took 1.968003273010254 seconds
[INFO 2019-03-13 17:06:49,673 cleverhans] Epoch 4 took 1.9837305545806885 seconds
[INFO 2019-03-13 17:06:51,824 cleverhans] Epoch 5 took 1.9669532775878906 seconds
[INFO 2019-03-13 17:06:53,966 cleverhans] Epoch 6 took 1.963310956954956 seconds
[INFO 2019-03-13 17:06:56,130 cleverhans] Epoch 7 took 1.981607437133789 seconds
[INFO 2019-03-13 17:06:58,369 cleverhans] Epoch 8 took 2.057114362716675 seconds
[INFO 2019-03-13 17:07:00,531 cleverhans] Epoch 9 took 1.9778735637664795 seconds


Generating new data points:
BATCH #1
Substitute training epoch:
num_devices:  1


[INFO 2019-03-13 17:07:45,307 cleverhans] Epoch 0 took 4.025018930435181 seconds
[INFO 2019-03-13 17:07:49,551 cleverhans] Epoch 1 took 3.7059497833251953 seconds
[INFO 2019-03-13 17:07:53,927 cleverhans] Epoch 2 took 3.8338005542755127 seconds
[INFO 2019-03-13 17:07:58,198 cleverhans] Epoch 3 took 3.7349836826324463 seconds
[INFO 2019-03-13 17:08:02,568 cleverhans] Epoch 4 took 3.8406639099121094 seconds
[INFO 2019-03-13 17:08:06,853 cleverhans] Epoch 5 took 3.7476813793182373 seconds
[INFO 2019-03-13 17:08:11,217 cleverhans] Epoch 6 took 3.817751407623291 seconds
[INFO 2019-03-13 17:08:15,448 cleverhans] Epoch 7 took 3.700002670288086 seconds
[INFO 2019-03-13 17:08:19,832 cleverhans] Epoch 8 took 3.841186285018921 seconds
[INFO 2019-03-13 17:08:24,087 cleverhans] Epoch 9 took 3.719571828842163 seconds


Generating new data points:
BATCH #2
Substitute training epoch:
num_devices:  1


[INFO 2019-03-13 17:09:55,338 cleverhans] Epoch 0 took 8.0779709815979 seconds
[INFO 2019-03-13 17:10:04,643 cleverhans] Epoch 1 took 7.551911115646362 seconds
[INFO 2019-03-13 17:10:13,702 cleverhans] Epoch 2 took 7.765395164489746 seconds
[INFO 2019-03-13 17:10:22,626 cleverhans] Epoch 3 took 7.608691453933716 seconds
[INFO 2019-03-13 17:10:31,448 cleverhans] Epoch 4 took 7.532013177871704 seconds
[INFO 2019-03-13 17:10:40,228 cleverhans] Epoch 5 took 7.46172571182251 seconds
[INFO 2019-03-13 17:10:48,998 cleverhans] Epoch 6 took 7.503694534301758 seconds
[INFO 2019-03-13 17:10:57,920 cleverhans] Epoch 7 took 7.635071039199829 seconds
[INFO 2019-03-13 17:11:06,720 cleverhans] Epoch 8 took 7.5040178298950195 seconds
[INFO 2019-03-13 17:11:15,522 cleverhans] Epoch 9 took 7.5153584480285645 seconds


Generating new data points:
BATCH #3
Substitute training epoch:
num_devices:  1


[INFO 2019-03-13 17:14:21,392 cleverhans] Epoch 0 took 15.882344245910645 seconds
[INFO 2019-03-13 17:14:39,985 cleverhans] Epoch 1 took 15.145636796951294 seconds


In [112]:
adversary_training_labels

array([17, 22, 41, ..., 45, 32, 37])

In [0]:
# Evaluate the substitute model on clean test examples
eval_params = {'batch_size': batch_size}
acc = model_eval(sess, x, y, preds_sub, x_test, y_test, args=eval_params)
accuracies['sub'] = acc

In [0]:
# Initialize the Fast Gradient Sign Method (FGSM) attack object.
fgsm_par = {'eps': 0.3, 'ord': np.inf, 'clip_min': 0., 'clip_max': 1.}
fgsm = FastGradientMethod(model_sub, sess=sess)


In [0]:
# Craft adversarial examples using the substitute
eval_params = {'batch_size': batch_size}
x_adv_sub = fgsm.generate(x, **fgsm_par)

In [0]:
# Evaluate the accuracy of the "black-box" model on adversarial examples
accuracy = model_eval(sess, x, y, model.get_logits(x_adv_sub),
                    x_test, y_test, args=eval_params)
print('Test accuracy of oracle on adversarial examples generated '
    'using the substitute: ' + str(accuracy))
accuracies['bbox_on_sub_adv_ex'] = accuracy